In [93]:
import gurobipy as grb
from gurobipy import GRB
import scipy.sparse as spr
import numpy as np
import random
import matplotlib.pyplot as plt
#from sympy import symbols, Rational
from IPython.display import display, Math, Markdown
import numpy.ma as ma
import scipy as sp

Let $\alpha_{ij}, \gamma_{ij}\in \mathbb{R};$ $0=t^0<t^1<\dots, t^K$ tax thresholds and $0 \leq \tau^0<\tau^1<\dots, \tau^K$ progressive tax rates. Compute recursively $n^k$
as $n^0 = 0, \ n^k := n^{k-1} + (1-\tau^k) (t^k -t^{k-1})$ and then $N^k := n^k - (1-\tau^k) t^k .$



We have 
$$ \mathcal{U}_{ij}(v) = \alpha_{ij} + \min_{k \in [0,K]} \{N^k + (1-\tau^k)(\gamma_{ij} -v)\}$$
$$ \mathcal{V}_{ij}(u) = \gamma_{ij} + \min_{k \in [0,K]} \left\{ \frac{N^k + \alpha_{ij} -u}{1-\tau^k}\right\}$$
$$ D_{ij}(u,v) = \max_{k \in [0,K]} \left\{ \frac{u-\alpha_{ij} - N^k + (1-\tau^k) (v_j - \gamma_{ij})}{2-\tau_k}\right\}$$

In [94]:
class OneToOneITU():
    def __init__(self, n, m, parameters = (None,None) , lbs=(0, 0)):
        self.n = n
        self.m = m
        self.lb_U, self.lb_V = lbs
     
       
        self.α_ij = parameters[0]
        self.γ_ij = parameters[1]
        self.t_k, self.τ_k = parameters[2]

        self.n_k = np.zeros(self.t_k.size)
        for m in range(1,self.t_k.size):
            self.n_k[m] = self.n_k[m -1] + (1-self.τ_k[m-1])* (self.t_k[m] - self.t_k[m-1])
        self.N_k = self.n_k - (1-self.τ_k) * self.t_k  
     

    def D_ij(self, u_i, v_j):
        D = np.max( (u_i[:,None,None] - self.α_ij[:,:,None] - self.N_k[None,None,:]+  
                     (1 - self.τ_k[None,None,:] )*(v_j[None,:,None] -self.γ_ij[:,:,None] ))/ 
                     (2 - self.τ_k[None,None,:]),      
                     axis= 2)
        return D
 
    def get_U_ij(self, v_j, i_idx ,j_idx = None):
        if j_idx is None:
            return self.α_ij[i_idx] +   np.min(self.N_k[None,None,:] + (1- self.τ_k[None,None,:])  * ( self.γ_ij[i_idx,:,None] - v_j[None, :, None] ), axis= 2)
        else:
            return self.α_ij[i_idx,j_idx] +   np.min(self.N_k[None,None,:] + (1- self.τ_k[None,None,:])  * ( self.γ_ij[i_idx,j_idx,None] - v_j[None, :, None] ), axis= 2)

    def get_V_ij(self, u_i, j_idx ,i_idx = None):
        if i_idx is None:
            return self.γ_ij[:,j_idx] + np.min ( (self.N_k[None,None,:] + self.α_ij[:,j_idx,None] - u_i[:,None,None])/(1-self.τ_k[None,None,:]) , axis = 2) 
        else:
           
            return self.γ_ij[i_idx,j_idx] + np.min ( (self.N_k[None,None,:] + self.α_ij[i_idx,j_idx,None] - u_i[:,None,None])/(1-self.τ_k[None,None,:]) , axis = 2) 

# Algorithms

In [95]:
def check_primal_feas(self, mu_ij):
    print(f"i: { np.all(np.sum(mu_ij,axis=1) <= 1)}, j:  {np.all(np.sum(mu_ij,axis=0) <= 1) }")
    print(f"#matched: {int(np.sum(mu_ij))} over {np.minimum(self.n,self.m)}")

OneToOneITU.check_primal_feas = check_primal_feas

def check_IR(self,mu_ij,U_i,V_j, output = False):
    IR_i = np.sum((U_i - self.lb_U)* (1- np.sum(mu_ij,axis=1)))
    IR_j = np.sum((V_j - self.lb_V) * (1 - np.sum(mu_ij,axis=0))) 
    print(f"i: { IR_i}, j:  {IR_j }")
    if output is True:
        IR_i = (U_i - self.lb_U)* (np.sum(mu_ij,axis=1) - 1) >= 0 
        IR_j = (V_j - self.lb_V) * (np.sum(mu_ij,axis=0) - 1) >= 0
        return IR_i, IR_j
    
OneToOneITU.check_IR = check_IR

def check_CS(self,U_i,V_j, output = None):
    CS = np.minimum(self.D_ij(U_i,V_j),0)
    if output:
        return CS, np.where(CS < 0 )
    print(f"min_ij D_ij : {(self.D_ij(U_i,V_j)).min()} ")
    
OneToOneITU.check_CS = check_CS 

def check_all(self,eq):
    mu_ij, U_i, V_j = eq 
    print("FEAS")
    self.check_primal_feas(mu_ij)
    print("________________")
    print("ε-CS")
    self.check_CS(U_i, V_j )
    print("________________")
    print("IR")
    self.check_IR(mu_ij, U_i, V_j )

OneToOneITU.check_all = check_all



In [96]:


def forward_auction(self, data = None, tol_ε = None):
    if data is None:
        V_j = np.ones(self.m) * self.lb_V
        mu_i = np.ones( self.n, dtype= int) * (self.m +1)
    else:
        mu_ij_01 = data[0]
        V_j = data[1]
        id_i, id_j = np.where(mu_ij_01 > 0)
        mu_i = np.ones(self.n, dtype= int) * (self.m+1)
        mu_i[id_i] = id_j

    unassigned_i = np.where(mu_i == self.m + 1)[0] 
    n_unassigned_i = len(unassigned_i)
    iter = 0

    while n_unassigned_i > 0:
        iter += 1
        U_ij = self.get_U_ij( V_j, unassigned_i)
        perm_unmatched = np.all(U_ij <= self.lb_U+tol_ε, axis = 1)#np.all(U_ij <= self.lb_U, axis = 1)
        
        if np.any(perm_unmatched):
            mu_i[unassigned_i] += (self.m - mu_i[unassigned_i]) * perm_unmatched
        else:
            ### bidding phase
            U_ij = np.concatenate((U_ij, self.lb_U * np.ones((n_unassigned_i,1))), axis = 1)
            j_i = np.argmax(U_ij, axis= 1)
            masked_U_ij = np.where(j_i[:,None] == np.arange(self.m+1)[None, :], -np.inf,U_ij)
            w_i = np.max(masked_U_ij, axis=1)
            j_i_unique = np.unique(j_i)
            offers = np.where(j_i[:,None] == j_i_unique[None,:], 
                              self.get_V_ij(w_i - tol_ε,j_i_unique[None,:],unassigned_i[:,None]), 
                              np.nan)
       
            ### assignment phase
            i_j_among_unass = np.nanargmax(offers, axis = 0)
            i_j = unassigned_i[ i_j_among_unass]
            best_offer_j = offers[i_j_among_unass, np.arange(len(j_i_unique))]

            # modify solution
            mu_i[  np.any( mu_i[:,None] == j_i_unique , axis = 1) ] = self.m +1
            mu_i[i_j] = j_i_unique
            V_j[j_i_unique] = best_offer_j #- V_j[j_i_unique] + tol_ε / self.A_ij[i_j,j_i_unique] #np.maximum(best_offer_j - V_j[j_i_unique], tol_ε/self.A_ij[i_j,j_i_unique])
        
        unassigned_i = np.where(mu_i == self.m + 1)[0]
        n_unassigned_i = len(unassigned_i)
        
    print(f"for: {iter}")
    matched_i =  mu_i < self.m
    U_i = np.ones(self.n) * self.lb_U
    U_i[matched_i] = self.get_U_ij(V_j, matched_i)[np.arange(matched_i.sum()), mu_i[matched_i]]
    mu_ij_01 = mu_i[:,None] == np.arange(self.m +1)
    
    return mu_ij_01[:,:-1], U_i, V_j
    


In [97]:

OneToOneITU.forward_auction = forward_auction
def reverse_auction(self, data = None, tol_ε = None):
    if data is None:
        U_i = np.ones(self.n) * self.lb_U
        mu_j = np.ones(self.m, dtype= int) * (self.n +1)
    else:
        mu_j_01 = data[0]
        U_i = data[1]
        id_i, id_j = np.where(mu_j_01 >0)
        mu_j = np.ones(self.m, dtype= int) * (self.n+1)
        mu_j[id_j] = id_i

    unassigned_j = np.where(mu_j == self.n + 1)[0] 
    n_unassigned_j = len(unassigned_j)
    iter = 0

    while n_unassigned_j > 0:
        iter += 1
        V_ij = self.get_V_ij( U_i , unassigned_j)
        perm_unmatched = np.all(V_ij <= self.lb_V+ tol_ε, axis = 0) 
        
        if np.any(perm_unmatched):
            mu_j[unassigned_j] += (self.n - mu_j[unassigned_j]) * perm_unmatched
        
        else:
            ### bidding phase
            V_ij = np.concatenate((V_ij, self.lb_V * np.ones((1,n_unassigned_j))), axis = 0)
            i_j = np.argmax(V_ij, axis= 0)
            masked_V_ij = np.where( np.arange(self.n+1)[:, None] == i_j[None,:] , -np.inf,V_ij)
            β_j = np.max(masked_V_ij, axis=0)
            i_j_unique = np.unique(i_j)
            offers = np.where(i_j_unique[:,None] == i_j[None,:], 
                              self.get_U_ij(β_j - tol_ε,i_j_unique[:,None] ,unassigned_j[None,:]), 
                              np.nan)

            ### assignment phase
            j_i_among_unass = np.nanargmax(offers, axis = 1)
            j_i = unassigned_j[j_i_among_unass]
            best_offer_i = offers[np.arange(len(i_j_unique)),j_i_among_unass]
            # modify solution
            mu_j[ np.any( mu_j[:,None] == i_j_unique , axis = 1) ] = self.n +1
            mu_j[j_i] = i_j_unique
            U_i[i_j_unique] = best_offer_i
                                        
        unassigned_j = np.where(mu_j == self.n + 1)[0]
        n_unassigned_j = len(unassigned_j)
        
    print(f"rev: {iter}") 
    matched_j =  mu_j < self.n
    V_j = np.ones(self.m) * self.lb_V
    V_j[mu_j < self.n] = self.get_V_ij(U_i, matched_j )[mu_j[matched_j],np.arange(matched_j.sum()) ]
    mu_j_01 = np.arange(self.n +1)[:,None] == mu_j[None,:]

    return mu_j_01[:-1,:], U_i, V_j


In [98]:

OneToOneITU.reverse_auction = reverse_auction
def drop_for_scaling(self, mu_ij_01, U_i,V_j, tol_ε, side = 1):
    if side == 1:
        violations_j = np.any( V_j[None,:] + tol_ε <  self.get_V_ij(U_i  ,np.arange(self.m)), axis=  0)
        #violations_j = np.any( V_j[None,:] <  self.get_V_ij(U_i + tol_ε ,np.arange(self.m)), axis=  0)
        # print(violations_j.sum())
        mu_ij_01[:,violations_j] = 0

        print(f"dropped: {violations_j.sum()}")
        # violations_i = np.any( U_i[:,None]+ tol_ε <  self.get_U_ij(V_j,np.arange(self.n)), axis=  1)
        # print(violations_i.sum())
        # mu_ij_01[violations_i,:] = 0

      
    else:
        violations_i = np.any( U_i[:,None] + tol_ε <  self.get_U_ij(V_j  ,np.arange(self.n)), axis=  1)
   
        mu_ij_01[violations_i,:] = 0

        print(f"dropped: {violations_i.sum()}")
        # violations_i = np.any( U_i[:,None]+ tol_ε <  self.get_U_ij(V_j,np.arange(self.n)), axis=  1)
        # print(violations_i.sum())
        # mu_ij_01[violations_i,:] = 0

    return mu_ij_01, U_i,V_j

In [99]:

OneToOneITU.drop_for_scaling = drop_for_scaling
def forward_backward_scaling(self, tol_ε ,tol_ε_obj, scaling_factor):
    mu_ij_01, u_i, v_j = self.forward_auction(tol_ε = tol_ε)

    while tol_ε > tol_ε_obj:
        tol_ε *=  scaling_factor
        # print("####")
        # print( tol_ε)
        print("#######")
        mu_ij_01, u_i, v_j = self.drop_for_scaling(mu_ij_01, u_i, v_j , tol_ε)
        mu_ij_01, u_i, v_j = self.reverse_auction((mu_ij_01, u_i), tol_ε= tol_ε)
        # tol_ε *=  scaling_factor
        #mu_ij_01, u_i, v_j = self.drop_for_scaling(mu_ij_01, u_i, v_j , tol_ε, side = 0)
        mu_ij_01, u_i, v_j = self.forward_auction((mu_ij_01, v_j ), tol_ε = tol_ε)
        self.check_CS(u_i,v_j)
        # print(np.all(self.check_CS(u_i,v_j,True)[0]>= - tol_ε_obj))

        if np.all(self.check_CS(u_i,v_j,True)[0]>= - tol_ε_obj):
             return mu_ij_01, u_i, v_j

    return mu_ij_01, u_i, v_j
OneToOneITU.forward_backward_scaling =forward_backward_scaling  

# Example

In [ ]:
n_i =  1000
m_j  = 1200
np.random.seed(123)
α_ij = np.zeros([n_i,m_j]) #np.random.choice([.25,.5,1,2,4], size= [n_i,m_j])
γ_ij= np.random.randint(0,5, size= [ n_i,1]) * np.random.randint(0,5, size= [ 1,m_j])  #np.random.randint(1,10000, size= [ n_i,1]) * np.random.randint(1,20, size= [ 1,m_j])
t_k = np.array([0, 9.701, 39.476, 84.201, 160.726, 204.101, 510.300])/100
tau_k =  np.array([.1,.12,   .22,    .24,    .32,     .35,     .37    ])

In [101]:
t_k

array([0.     , 0.09701, 0.39476, 0.84201, 1.60726, 2.04101, 5.103  ])

In [102]:
example_mkt = OneToOneITU(  n_i,m_j,parameters=(α_ij,γ_ij,(t_k,tau_k)))

In [103]:
example_mkt.get_U_ij(np.ones(1)* 10000,1,1)

array([[-8989.2]])

In [104]:
np.shape(example_mkt.get_V_ij(np.ones(n_i),np.ones(m_j, dtype=bool)))

: 

: 

Import algorithms

In [ ]:
# %run /Users/enzo-macmini/auction_alg_ITU/for_rev_scaling_ALGs.ipynb

In [ ]:
# np.shape(example_mkt.get_V_ij(u_i_noscaling,np.arange(example_mkt.m)))

In [ ]:
# eq_noscaling = mu_ij_01_noscaling, u_i_noscaling, v_j_noscaling = example_mkt.forward_auction(tol_ε= .1)
# # print(np.all( u_i_noscaling[:,None]+ 1>=  example_mkt.get_U_ij(v_j_noscaling,np.arange(example_mkt.n)), axis= 1 ))
# print(np.min(u_i_noscaling[:,None] -  example_mkt.get_U_ij(v_j_noscaling,np.arange(example_mkt.n))))
# print(np.min(v_j_noscaling[None,:] -  example_mkt.get_V_ij(u_i_noscaling,np.arange(example_mkt.m))))

In [ ]:
eq_scaling = mu_ij_01_scaling, u_i_scaling, v_j_scaling = example_mkt.forward_backward_scaling(1000,1e-13,1/2)

for: 1
#######
rev: 1
for: 1
min_ij D_ij : -6.398851226993865 
#######
rev: 1
for: 1
min_ij D_ij : -6.398851226993865 
#######
rev: 1
for: 1
min_ij D_ij : -6.398851226993865 
#######
rev: 1
for: 1
min_ij D_ij : -6.398851226993865 
#######
rev: 1
for: 1
min_ij D_ij : -6.398851226993865 
#######
rev: 20
for: 1
min_ij D_ij : -6.253912576687116 
#######
rev: 29
for: 1
min_ij D_ij : -4.707900306748466 
#######
rev: 18
for: 24
min_ij D_ij : -2.3964723926380374 
#######
rev: 41
for: 1
min_ij D_ij : -0.8764951136363638 
#######
rev: 14
for: 1
min_ij D_ij : -0.7548888036809817 
#######
rev: 1
for: 245
min_ij D_ij : -0.2995590490797546 
#######
rev: 113
for: 1
min_ij D_ij : -0.11531061170212767 
#######
rev: 1
for: 2
min_ij D_ij : -0.10542436079545454 
#######
rev: 4
for: 1
min_ij D_ij : -0.09617660984848485 
#######
rev: 1
for: 137
min_ij D_ij : -0.04229046416546362 
#######
rev: 94
for: 1
min_ij D_ij : -0.0072278474506578955 
#######
rev: 7
for: 2
min_ij D_ij : -0.006700638967856515 
#######
r

In [ ]:
example_mkt.check_CS(u_i_scaling, v_j_scaling)

min_ij D_ij : -9.780860511421625e-14 


In [ ]:
example_mkt.check_all((mu_ij_01_scaling, u_i_scaling, v_j_scaling))

FEAS
i: True, j:  True
#matched: 92 over 100
________________
ε-CS
min_ij D_ij : -9.780860511421625e-14 
________________
IR
i: 0.0, j:  0.0


In [ ]:
print(np.min(u_i_scaling[:,None] -  example_mkt.get_U_ij(v_j_scaling,np.arange(example_mkt.n))))
print(np.min(v_j_scaling[None,:] -  example_mkt.get_V_ij(u_i_scaling,np.arange(example_mkt.m))))

-1.6076029396572267e-13
-2.531308496145357e-13


In [ ]:
v_j_scaling.max()

6.0000000000002105

In [ ]:
np.mean( (example_mkt.γ_ij - v_j_scaling[None,:])[mu_ij_01_scaling>0] )

4.456521739130375

In [ ]:
u_i_scaling.mean()

2.832726546999965

In [ ]:
u_i_scaling

array([ 2.19806750e+00,  6.65012750e+00,  2.19806750e+00,  8.18256400e-01,
        4.13012750e+00,  2.19806750e+00,  4.13012750e+00,  8.18256400e-01,
        8.18256400e-01,  2.77777801e-14,  8.18256400e-01,  8.18256400e-01,
        1.09690035e-14, -9.50350909e-15,  8.18256400e-01,  4.13012750e+00,
        6.65012750e+00, -1.11022302e-14,  0.00000000e+00,  6.65012750e+00,
        8.18256400e-01,  4.13012750e+00,  2.19806750e+00,  6.65012750e+00,
        2.19806750e+00,  6.65012750e+00, -3.25073289e-14,  0.00000000e+00,
        8.18256400e-01,  4.13012750e+00,  6.65012750e+00,  6.65012750e+00,
        6.65012750e+00,  8.18256400e-01,  4.13012750e+00,  2.19806750e+00,
        8.18256400e-01,  6.65012750e+00, -5.12923037e-15,  4.13012750e+00,
        2.19806750e+00,  1.59872111e-15,  4.13012750e+00,  2.19806750e+00,
        2.19806750e+00,  2.19806750e+00,  2.19806750e+00,  6.65012750e+00,
        4.13012750e+00,  4.13012750e+00,  6.65012750e+00,  6.65012750e+00,
        4.13012750e+00,  